In [1]:
import os
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/AAPL")

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 150)


In [29]:
import pandas as pd

# Load normally (row 0 is the header)
valuation = pd.read_csv("AAPL_monthly_valuation_measures_cleaned.csv")

# Drop the first data row if it contains 'ttm' (non-date value)
valuation = valuation[~valuation["date"].str.lower().str.contains("ttm", na=False)]

# Drop the last two mostly-empty columns, if present
if valuation.shape[1] > 11:
    valuation = valuation.iloc[:, :11]

# Convert 'date' to datetime
valuation["date"] = pd.to_datetime(valuation["date"], errors="coerce")
valuation = valuation.dropna(subset=["date"])

# Clean numeric fields
for col in valuation.columns:
    if col not in ["date", "ticker"]:
        valuation[col] = (
            valuation[col].astype(str)
            .str.replace(",", "", regex=False)
            .replace("", pd.NA)
            .astype(float)
        )

# Normalize date
valuation["date"] = valuation["date"].dt.normalize()

# Sort and drop duplicate dates
valuation = valuation.sort_values("date").drop_duplicates(subset="date", keep="last")
# Filter for dates between 2014 and 2025 inclusive
valuation = valuation[
    (valuation["date"] >= "2014-01-01") & (valuation["date"] <= "2024-12-31")
]
# Drop unwanted columns
valuation = valuation.drop(columns=["EnterprisesValueRevenueRatio", "EnterprisesValueEBITDARatio"], errors='ignore')

print("✅ Valuation data cleaned and ready to merge.")


✅ Valuation data cleaned and ready to merge.


In [30]:
valuation.to_csv("AAPL_valuations.csv")

In [ ]:
import pandas as pd
import numpy as np
from dateutil import parser

# --- Load options data ---
opt = pd.read_csv("AAPL_options_all.csv")

# Manual renaming of critical columns to avoid double underscore issues
rename_map = {
    'last_trade_date_(edt)': 'last_trade_date',
    'expiry_date': 'expiry_date',
    'contract_name': 'contract_name',
    'open_interest': 'open_interest',
    'volume': 'volume',
    'implied_volatility': 'implied_volatility'
}
opt.rename(columns=rename_map, inplace=True)

# Strip and standardize all remaining column names
opt.columns = (
    opt.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[^\w]+', '_', regex=True)
    .str.replace(r'_+', '_', regex=True)
    .str.strip('_')
)

# --- Parse dates robustly ---
def parse_mixed_datetime(val):
    if pd.isna(val):
        return pd.NaT
    try:
        return pd.to_datetime(val, format='%m/%d/%Y %I:%M %p')  # Try US format first
    except Exception:
        try:
            return pd.to_datetime(val)  # Fallback to ISO/generic
        except Exception:
            return pd.NaT

opt['last_trade_date'] = opt['last_trade_date'].apply(parse_mixed_datetime)
opt['expiry_date'] = pd.to_datetime(opt['expiry_date'], errors='coerce')

# --- Filter and clean ---
opt = opt.dropna(subset=['last_trade_date', 'expiry_date'])
opt = opt[opt['contract_name'].str.contains("AAPL", na=False)]

# Convert numerics
opt['open_interest'] = pd.to_numeric(opt['open_interest'], errors='coerce')
opt['volume'] = pd.to_numeric(opt['volume'], errors='coerce')
opt['implied_volatility'] = (
    pd.to_numeric(opt['implied_volatility'].astype(str).str.replace('%', ''), errors='coerce') / 100
)

# Extract date, option type, DTE
opt['option_type'] = opt['contract_name'].str.extract(r'(\d+[CP])')[0].str[-1].map({'C': 'call', 'P': 'put'})
opt['dte'] = (opt['expiry_date'] - opt['last_trade_date']).dt.days
opt['date'] = opt['last_trade_date'].dt.floor('D')

def tag_dte_bucket(dte):
    if 0 <= dte <= 10:
        return 'short'
    elif 20 <= dte <= 40:
        return 'medium'
    elif 50 <= dte <= 150:
        return 'long'
    return None

opt['dte_bucket'] = opt['dte'].apply(tag_dte_bucket)
opt = opt[opt['dte_bucket'].notnull()]

# --- Done ---
print("✅ Cleaned options data and engineered sentiment features.")


✅ Cleaned options data and engineered sentiment features.


In [48]:
opt.to_csv("AAPL_options_vdx.csv")

In [23]:
os.getcwd()

'/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/AAPL'

In [22]:
import pandas as pd

# --- Load Datasets ---
daily_factors = pd.read_csv("merged_daily_factors_final.csv", parse_dates=["date"])
technicals = pd.read_csv("AAPL_2014_2024_technical_cleaned.csv", parse_dates=["date"])
valuation = pd.read_csv(
    "AAPL_monthly_valuation_measures_cleaned.csv",
    parse_dates=["date"],
    date_format="%m/%d/%Y",
)
# Optional: if 'date' column is still object type, parse explicitly
valuation["date"] = pd.to_datetime(valuation["date"], errors="coerce")
valuation = valuation.dropna(subset=["date"]).sort_values("date")
# Collect in list
dfs = [daily_factors, technicals, valuation]

# Normalize dates and remove timezones
# Normalize and remove timezone safely
for df in dfs:
    df["date"] = pd.to_datetime(df["date"])
    if df["date"].dt.tz is not None:
        df["date"] = df["date"].dt.tz_localize(None)
    df["date"] = df["date"].dt.normalize()

# Drop true duplicate dates (keep last seen)
for i in range(len(dfs)):
    dfs[i] = dfs[i].drop_duplicates(subset="date", keep="last")

# Merge everything into one unified DataFrame
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on="date", how="left", suffixes=('', '_dup'))

# Optional: Handle _dup columns if any


/var/folders/g9/jl77b7s511d2m50h489cqhwr0000gp/T/ipykernel_82382/1625377851.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  valuation["date"] = pd.to_datetime(valuation["date"], errors="coerce")


In [23]:
for i, df in enumerate(dfs):
    print(f"Dataset {i} preview:")
    print(df["date"].head(5))


Dataset 0 preview:
0   2014-01-02
1   2014-01-03
2   2014-01-06
3   2014-01-07
4   2014-01-08
Name: date, dtype: datetime64[ns]
Dataset 1 preview:
0   2014-01-01
1   2014-01-02
2   2014-01-05
3   2014-01-06
4   2014-01-07
Name: date, dtype: datetime64[ns]
Dataset 2 preview:
498   1985-08-31
497   1985-09-30
496   1985-10-31
495   1985-11-30
494   1985-12-31
Name: date, dtype: datetime64[ns]


In [24]:
import pandas as pd
import numpy as np

# --- Load Datasets ---
daily_factors = pd.read_csv("merged_daily_factors_final.csv", parse_dates=["date"])
technicals = pd.read_csv("AAPL_2014_2024_technical_cleaned.csv", parse_dates=["date"])
valuation = pd.read_csv(
    "AAPL_monthly_valuation_measures_cleaned.csv",
    parse_dates=["date"],
    date_format="%m/%d/%Y"
)

# --- Clean Technicals ---
def clean_technicals(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"[^\w]+", "_", regex=True)
        .str.replace(r"_+", "_", regex=True)
        .str.strip("_")
    )
    rename_map = {
        'rsi_rsi_14': 'rsi',
        'macd_macd_12_26_9': 'macd',
        'signal_macd_12_26_9': 'macd_signal',
        'macd_12_26_9_hist': 'macd_hist',
        'result_schaff_10_23_50_ema': 'schaff_trend',
        'result_w_acc_dist_n': 'acc_dist',
        'result_ultimate_7_14_28': 'ultimate_osc',
        'result_trix_14': 'trix',
        'result_williams_r_14': 'williams_r',
        'result_vol_roc_14': 'vol_roc',
        'result_vol_osc_12_26_points': 'vol_osc',
        'rel_vol_rel_vol_10_14': 'rel_vol',
        'rel_vig_rel_vig_10': 'rel_vig',
        'relvigsignal_rel_vig_10': 'rel_vig_signal',
        'rel_vig_10_hist': 'rel_vig_hist',
        'ravi_vdma_7_65_hist': 'ravi',
        'result_psar_0_02_0_2': 'psar',
        'result_on_bal_vol': 'obv',
        'result_price_roc_14': 'roc',
        'result_price_osc_12_26_ema_points': 'price_osc',
        'result_chande_mtm_9': 'chande_mtm',
        'result_cci_20': 'cci',
        'result_m_flow_14': 'money_flow',
        'result_momentum_14': 'momentum',
        'result_med_price_14': 'median_price',
        'result_std_dev_14_2_ma': 'std_dev',
        'result_hist_vol_10_252_1': 'hist_vol',
        'result_perf_idx_120_spy': 'perf_idx_spy',
        'result_beta_20_spy': 'beta_spy',
    }
    return df.rename(columns=rename_map)

technicals = clean_technicals(technicals)

# --- Prepare Valuation (monthly → daily ffill) ---
valuation["date"] = pd.to_datetime(valuation["date"], errors="coerce")
valuation = valuation.dropna(subset=["date"])

# --- Diagnostic: Duplicated dates
dup_counts = valuation['date'].value_counts()
duplicate_dates = dup_counts[dup_counts > 1]
print(f"⚠️ Found {len(duplicate_dates)} duplicate date(s) in valuation data.")
if not duplicate_dates.empty:
    print(duplicate_dates)

# --- Safely aggregate duplicates by averaging numerics
valuation = (
    valuation.groupby("date", as_index=False)
             .mean(numeric_only=True)
)

# --- Resample to daily frequency
valuation_daily = (
    valuation.set_index("date")
             .sort_index()
             .resample("D")
             .ffill()
             .reset_index()
)

# --- Strip timezones from all date columns
for df_ in [daily_factors, technicals, valuation_daily, options]:
    if 'date' in df_.columns:
        df_["date"] = pd.to_datetime(df_["date"]).dt.tz_localize(None)
    else:
        print(f"⚠️ Skipped timezone strip: 'date' column missing in {type(df_)}")

# --- Merge All Datasets ---
df = (
    daily_factors
    .merge(technicals, on="date", how="outer", suffixes=('', '_tech'))
    .merge(valuation_daily, on="date", how="left", suffixes=('', '_val'))
    .sort_values("date")
    .reset_index(drop=True)
)

# --- Final Check ---
print("✅ Unified dataset ready.")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()[:10]}... (+{len(df.columns) - 10} more)")


⚠️ Found 0 duplicate date(s) in valuation data.
⚠️ Skipped timezone strip: 'date' column missing in <class 'pandas.core.frame.DataFrame'>
✅ Unified dataset ready.
Date range: 2014-01-01 to 2025-01-01
Shape: (5559, 233)
Columns: ['date', 'ff3_mkt_rf', 'ff3_smb', 'ff3_hml', 'ff3_rf', 'ff3_mom', 'ff5_mkt_rf', 'ff5_smb', 'ff5_hml', 'ff5_rmw']... (+223 more)


/var/folders/g9/jl77b7s511d2m50h489cqhwr0000gp/T/ipykernel_82382/2681465812.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  valuation["date"] = pd.to_datetime(valuation["date"], errors="coerce")


In [25]:
df.to_csv("AAPL_unified_dataset.csv", index=False)


In [11]:
# Load the Parquet file
df_parquet = pd.read_parquet("AAPL_unified_dataset.parquet")

# Save to CSV for inspection
df_parquet.to_csv("AAPL_unified_dataset.csv", index=False)

print("📄 Saved: AAPL_unified_dataset.csv")


📄 Saved: AAPL_unified_dataset.csv
